In [15]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=150, return_messages=True)


def add_message(input, output):
    memory.save_context({'input': input}, {'output': output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi, I'm Flynn. I live in South Korea.", "Wow, That's so cool!")

In [16]:
get_history()

{'history': [HumanMessage(content="Hi, I'm Flynn. I live in South Korea."),
  AIMessage(content="Wow, That's so cool!")]}

In [17]:
add_message("South Korea is so pretty.", "I wish I could go!")

In [18]:
get_history()

{'history': [HumanMessage(content="Hi, I'm Flynn. I live in South Korea."),
  AIMessage(content="Wow, That's so cool!"),
  HumanMessage(content='South Korea is so pretty.'),
  AIMessage(content='I wish I could go!')]}

In [19]:
add_message("How far is Koread from Argentina?", "I don't know! But I think it's super far!")
get_history()

{'history': [HumanMessage(content="Hi, I'm Flynn. I live in South Korea."),
  AIMessage(content="Wow, That's so cool!"),
  HumanMessage(content='South Korea is so pretty.'),
  AIMessage(content='I wish I could go!'),
  HumanMessage(content='How far is Koread from Argentina?'),
  AIMessage(content="I don't know! But I think it's super far!")]}

In [21]:
add_message("What do you want to see at South Korea?", "South Korea has many places to visit, including palaces, villages, and other attractions: Gyeongbokgung Palace, Bukchon Hanok Village, Namsan Tower, and many more! I want to see them all!")
get_history()

{'history': [SystemMessage(content="Flynn introduces himself as living in South Korea, and the AI expresses admiration for the country. Flynn mentions how pretty South Korea is, and the AI expresses a desire to visit. When Flynn asks about the distance between South Korea and Argentina, the AI admits it doesn't know but assumes it's far."),
  HumanMessage(content='What do you want to see at South Korea?'),
  AIMessage(content='South Korea has many places to visit, including palaces, villages, and other attractions: Gyeongbokgung Palace, Bukchon Hanok Village, Namsan Tower, and many more! I want to see them all!'),
  HumanMessage(content='What do you want to see at South Korea?'),
  AIMessage(content='South Korea has many places to visit, including palaces, villages, and other attractions: Gyeongbokgung Palace, Bukchon Hanok Village, Namsan Tower, and many more! I want to see them all!')]}

In [22]:
add_message("And what do you want to eat in South Korea?", "I want to eat Kimchi, Bibimbap, Bulgogi, and many other Korean dishes. I heard they are delicious!")
get_history()

{'history': [SystemMessage(content="Flynn introduces himself as living in South Korea, and the AI expresses admiration for the country, mentioning how pretty it is. The AI expresses a desire to visit South Korea and lists some attractions it wants to see. When Flynn asks about the distance between South Korea and Argentina, the AI admits it doesn't know but assumes it's far."),
  HumanMessage(content='What do you want to see at South Korea?'),
  AIMessage(content='South Korea has many places to visit, including palaces, villages, and other attractions: Gyeongbokgung Palace, Bukchon Hanok Village, Namsan Tower, and many more! I want to see them all!'),
  HumanMessage(content='And what do you want to eat in South Korea?'),
  AIMessage(content='I want to eat Kimchi, Bibimbap, Bulgogi, and many other Korean dishes. I heard they are delicious!')]}